In [1]:
import ROOT
import edm4hep
from podio import root_io
from math import sqrt

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x745bad0
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.gInterpreter.Declare("#include <marlinutil/HelixClass_double.h>")
# ROOT.gInterpreter.Declare("#include <marlinutil/GeometryUtil.h>")

True

In [4]:
ROOT.ildStyle.SetOptStat(0)

In [5]:
# Meh would require dd4hep
# Bz = ROOT.MarlinUtil.getBzAtOrigin()
Bz = 2
print(Bz)

2


In [6]:
# reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.10.REC_edm4hep.root")
# reader = root_io.Reader("E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0.1000.REC_edm4hep.root")
reader = root_io.Reader("rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500106.P4f_sw_sl.eL.pR.n000.d_dstm_15176_0.edm4hep.root")

In [7]:
events = reader.get("events")


In [8]:
# TODO: define beam_cone, lumi cal clearance whatever already up here and make a custom binning with one bin each in the dead parts
# caveat: for the underlay distribution this is not nice

beam_cone = 0.11 / ROOT.Math.Pi()

h_e_theta = ROOT.TH1D("h_e_theta", ";#theta [#pi rad]", 30, 0, 1)
h_total_theta = ROOT.TH1D("h_total_theta", ";#theta [#pi rad]", 30, 0, 1)
h_pass_theta = ROOT.TH1D("h_pass_theta", ";#theta [#pi rad]", 30, 0, 1)
h_total_pt = ROOT.TH1D("h_total_pt", ";p_{T} [GeV]", 62, 0, 124)
h_pass_pt = ROOT.TH1D("h_pass_pt", ";#p_{T} [GeV]", 62, 0, 124)

In [9]:
all = 0
found = 0
all_acc = 0
for event in events:
    if all >= 10000:
        break
    all += 1
    mcparticles = event.get("MCParticlesSkimmed")
    # search first genStat 1 electron
    for mcp in mcparticles:
        if (abs(mcp.getPDG()) == 11 and mcp.getGeneratorStatus() == 1):
            break
    mom = mcp.getMomentum()
    mom_v = ROOT.TVector3(mom.x, mom.y, mom.z)
    # if (abs(mom_v.CosTheta()) >= 0.99):
    #     pass
    h_total_theta.Fill(mom_v.Theta() / ROOT.Math.Pi())
    h_total_pt.Fill(mom_v.Perp())
    if (mcp.getCharge() == -1.0):
        h_e_theta.Fill(mom_v.Theta() / ROOT.Math.Pi())
    if (mom_v.Theta() / ROOT.Math.Pi() > beam_cone and mom_v.Theta() / ROOT.Math.Pi() < 1 - beam_cone):
        all_acc += 1

    mctruthrecolinks = event.get("MarlinTrkTracksMCTruthLink")
    # meh need to do the whole logic as in my marlin pidutil
    # lets just care about tracks its simpler
    # argh this can also have more than one track per electron
    for link in mctruthrecolinks:
        if link.getSim() == mcp and link.getWeight() > 0.99:
            # print(f"found! with weight: {link.getWeight()}")
            found += 1
            h_pass_theta.Fill(mom_v.Theta() / ROOT.Math.Pi())
            h_pass_pt.Fill(mom_v.Perp())
            break
print(f"track reco eff: {found / all}")
print(f"track reco eff only in acceptance: {found / all_acc}")

track reco eff: 0.9801
track reco eff only in acceptance: 1.0031729785056296


In [10]:
eff_theta = ROOT.TEfficiency(h_pass_theta, h_total_theta).CreateGraph()
eff_pt = ROOT.TEfficiency(h_pass_pt, h_total_pt).CreateGraph()

In [11]:
u_theta = h_e_theta.Clone()
u_theta.Scale(7 / u_theta.Integral())
u_theta.SetLineColor(ROOT.kGray)
u_theta.SetMarkerColor(ROOT.kGray)
u_theta.SetFillColor(ROOT.kGray)

u_pt = h_total_pt.Clone()
u_pt.Scale(7 / u_pt.Integral())
u_pt.SetLineColor(ROOT.kGray)
u_pt.SetMarkerColor(ROOT.kGray)
u_pt.SetFillColor(ROOT.kGray)

In [12]:
t = ROOT.TLatex()
t.SetTextSize(30)
t.SetTextFont(63)

In [13]:
c_eff_theta = ROOT.TCanvas()
eff_theta.SetTitle(";#theta_{e^{#pm}} [#pi rad]; e^{#pm} track reco efficiency")
u_theta.Draw("hist")
eff_theta.Draw("p")
u_theta.GetYaxis().SetRangeUser(0, 1.05)
u_theta.GetXaxis().SetRangeUser(0, 1.0)
# ll = ROOT.TLine(beam_cone, 0, beam_cone, 1.05)
# ll.SetLineColor(ROOT.kRed)
# lr = ROOT.TLine(1 - beam_cone, 0, 1 - beam_cone, 1.05)
# lr.SetLineColor(ROOT.kRed)
# ll.Draw()
# lr.Draw()
l = ROOT.TLegend(0.425, 0.5, 0.85, 0.7)
l.AddEntry(eff_theta, "e^{#pm} tracking efficiency")
# l.AddEntry(ll, "detector acceptance")
l.AddEntry(u_theta, "electron distribution #times 7")
l.Draw()
t.DrawLatex(.4, .75, "e^{+}e^{-}#rightarrow e^{#pm}#nu W")
t.DrawLatexNDC(0.15, 0.93935, "ILD #font[52]{work in progress}")
# c_eff_theta.BuildLegend(0.3, 0.3, 0.9, 0.6, "foo")
c_eff_theta.Draw()
c_eff_theta.SaveAs("plots/trk_eff_ild/eff_theta.pdf")

Info in <TCanvas::Print>: pdf file plots/trk_eff_ild/eff_theta.pdf has been created


In [14]:

c_eff_pt = ROOT.TCanvas()
eff_pt.SetTitle(";p_{T,e^{#pm}} [GeV]; e^{#pm} track reco efficiency")
u_pt.Draw("hist")
eff_pt.Draw("p")
u_pt.GetYaxis().SetRangeUser(0, 1.05)
u_pt.GetXaxis().SetRangeUser(0, 125.)
# ll = ROOT.TLine(beam_cone, 0, beam_cone, 1.05)
# ll.SetLineColor(ROOT.kRed)
# lr = ROOT.TLine(1 - beam_cone, 0, 1 - beam_cone, 1.05)
# lr.SetLineColor(ROOT.kRed)
# ll.Draw()
# lr.Draw()
l = ROOT.TLegend(0.3, 0.4, 0.8, 0.6)
l.AddEntry(eff_pt, "e^{#pm} tracking efficiency")
# l.AddEntry(ll, "detector acceptance")
l.AddEntry(u_pt, "electron distribution #times 7")
l.Draw()
t.DrawLatexNDC(.325, .6, "e^{+}e^{-}#rightarrow e^{#pm}#nu W")
t.DrawLatexNDC(0.15, 0.93935, "ILD #font[52]{work in progress}")
# c_eff_pt.BuildLegend(0.3, 0.3, 0.9, 0.6, "foo")
c_eff_pt.Draw()
c_eff_pt.SaveAs("plots/trk_eff_ild/eff_pt.pdf")

Info in <TCanvas::Print>: pdf file plots/trk_eff_ild/eff_pt.pdf has been created
